In [ ]:
!pip install graphrole

In [19]:
import pandas as pd
import snap
from src.load_and_save import load_networkx_directed_graph, save_list, load_list
from graphrole import RecursiveFeatureExtractor, RoleExtractor

In [20]:
spreading_path = "data/spreading.edgelist"

features_spreading = "data/features_spreading.csv"
features_followers = "data/features_followers.csv"

In [21]:
spreading_graph = load_networkx_directed_graph(spreading_path, weighted=True)

FileNotFoundError: [Errno 2] No such file or directory: 'data/spreading.edgelist'

In [ ]:
spreading_feature_extractor = RecursiveFeatureExtractor(spreading_graph)
spreading_features = spreading_feature_extractor.extract_features()
spreading_features.to_csv(features_spreading)

In [ ]:
dummy = pd.DataFrame(
    [ [1, 2, 3],
     [2, 3, 4],
     [1, 2, 3]], columns=["1", "2", "3"])
dummy

In [ ]:
role_extractor = RoleExtractor(n_roles=None)
role_extractor.extract_role_factors(dummy)
role_extractor.roles

In [ ]:
graphrole.__file__

# Adding extra features

In [ ]:
Nodes, _ = spreading_features.GetBetweennessCentr(1.0, False)
betweenessCentrList = []
for node in Nodes:
    betweenessCentrList.append(Nodes[node])

In [ ]:
KCorenessList = getKcorenessForEachNode()

# Save to file